# Recomendador de libros basado en generos literarios

### Autores: 

Julián Andrés Muñoz Montoya, julian.munozm@udea.edu.co

Santiago Alexis Sánchez Zuleta, salexis.sanchez@udea.edu.co



El siguiente recomendador de contenidos, hace recomendaciones basado en los géneros favoritos del usuario. Nuestro recomendador predice un mayor rating para aquellos libros que sean de un genero preferido por el usuario. En nuestra base de datos, los libros tienen tags, los cuales son análogos a los géneros.

La construcción de este recomendador consta de 3 etapas. 

En la primera etapa aplicamos el algoritmo Alternating Least Squares (ALS), sobre la tabla de ratings, la cual es de la siguiente forma.

| ID usuario    | ID Libro      | Rating|
| ------------- |:-------------:| -----:|
|      15       |    518285     |   4   |

El libro con el ID 518285 podría ser el señor de los anillos, entonces veamos que libros le podemos recomendar al usuario 15.

Al aplicar el algoritmo ALS, obtenemos la siguiente tabla.

| ID usuario    | ID Libro      | Rating| Predicción |
| ------------- |:-------------:|:-----:| ----------:|
|    155884     |    518285     |   4   |    3.89    |

Del algoritmo también podemos obtener las N recomendaciones top para cada usuario, por ejemplo con N = 3. Estas recomendaciones tienen un rating predecido por nuestro sistema.


| ID usuario | ID Libro| Rating predecido|
| -----      |  :-----:|         -------:|
|         148|     1400|                5|
|         148|       52|              4.6|
|         148|     6634|              4.4|
|         564|     1188|              4.2|
|         564|     6546|              3.6|
|         564|       39|              3.4|

En la segunda etapa tomamos estas recomendaciones y ejecutamos el siguiente proceso: 

1. Tomamos las N recomendaciones para el usuario i.
2. Extraemos los tags de estos libros.
3. Agrupamos los libros por tags y sacamos el promedio para los K tags preferidos por usuario.
4. Ordenamos los K tags preferidos por el usuario por su rating promedio, este rating promedio, será el rating de cada tag.
5. Tomamos nuevamente las N recomendaciones para el usuario i y al rating de cada libro j con el tag k, le sumamos el rating del usuario i para el tag k.

Sea i un usuario, j un libro recomendado al usuario i y k el tag del libro j.

* rating_pesado = rating_predecido(i, j) + tag_rating(i, k)

Nuevamente tenemos una tabla con usuarios, libros y ratings pesados.

| ID usuario | ID Libro| Rating pesado   |
| -----      |  :-----:|         -------:|
|         321|     1400|                5|
|         564|     6546|              4.8|
|         564|     1188|              4.5|
|         654|     1188|              4.9|

Note que en la tabla anterior a esta, para el usuario 564, el rating del libro 6546 era menor que el rating de este usuario para el libro 1188, sin embargo con nuestro rating pesado, el libro 6564 tiene mas peso que el libro 1188.

Finalmente en la tercera etapa aplicamos el algoritmo ALS sobre nuestros ratings pesados.

| ID usuario    | ID Libro      | Rating predecido |
| ------------- |:-------------:|            -----:|
|      15       |    418589     |            4.6   |

En este caso el libro 418589 es Juego de tronos de la serie de novelas canción de hielo y fuego es nuestra mejor recomendación para el usuario 15, el genero de este libro es alta fantasia al igual que El señor de los anillos, el cual tenia el mayor rating para el usuario 15.

In [1]:
import org.apache.spark.sql.types.{StructType, StructField}
import org.apache.spark.sql.functions.{desc, mean, col}
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import org.apache.spark.sql.types.IntegerType

### Dataset

Nuestro dataset goodbooks-10k contiene 10.000 libros con un millón de ratings.

En la etapa uno utilizamos la tabla ratings, en la etapa 2, las tablas tags, book_tags y en la etapa final utilizamos la tabla books, describiremos brevemente cada tabla.

* Ratings: esta tabla contiene todos los ratings de los usuarios.

* Tags: esta tabla contiene el id de cada tag y su nombre.

* Book Tags: esta tabla contiene los tags asociados a cada libro, es este dataset cada libro tiene 100 tags diferentes.

* Books: esta tabla contiene los metadatos del libro, como el nombre, el autor, año de publicación editorial y entre otros, de esta tabla tomamos solamente el nombre para mostrar al final las recomendaciones top para varios usuarios.

Dataset sacado de: https://www.kaggle.com/zygmunt/goodbooks-10k

Este recomendador fue creado con Spark 2.3.1 con el lenguaje Scala versión 2.12.7 y con el kernel Apache Toree versión 0.3.0.

## Etapa 0: Preprocesamiento

Primero hacemos un preprocesamiento del dataset para convertir las tablas en data frames.

In [8]:
val books = spark.read.format("csv").option("header", "true").load("./goodbooks-10k/books.csv")
//books = books.withColumnRenamed("id", "books_id")
                        
val book_tags1 = spark.read.format("csv").option("header", "true").load("goodbooks-10k/book_tags.csv")
val book_tags = book_tags1.withColumnRenamed("goodreads_book_id", "book_id")
val ratings1 = spark.read.format("csv").option("header", "true").load("goodbooks-10k/ratings.csv")
val ratings2 = ratings1.withColumn("user_id", $"user_id".cast(IntegerType))
                        .withColumn("book_id", $"book_id".cast(IntegerType))
                        .withColumn("rating", $"rating".cast(IntegerType))
val rawRatings = ratings2.select("user_id", "book_id", "rating")
val tags=spark.read.format("csv").option("header", "true").load("goodbooks-10k/tags.csv")
val to_read = spark.read.format("csv").option("header", "true").load("goodbooks-10k/to_read.csv")

books = [id: string, book_id: string ... 21 more fields]
book_tags1 = [goodreads_book_id: string, tag_id: string ... 1 more field]
book_tags = [book_id: string, tag_id: string ... 1 more field]
ratings1 = [book_id: string, user_id: string ... 1 more field]
ratings2 = [book_id: int, user_id: int ... 1 more field]
rawRatings = [user_id: int, book_id: int ... 1 more field]
tags = [tag_id: string, tag_name: string]
to_read = [user_id: string, book_id: string]


[user_id: string, book_id: string]

Para nuestro recomendador solo tomaremos las 3 siguientes tablas del dataset. RawRatings quiere decir que nuestros ratings no han sido normalizados.

In [9]:
rawRatings.show(1)
book_tags.show(1)
tags.show(1)

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|    314|      1|     5|
+-------+-------+------+
only showing top 1 row

+-------+------+------+
|book_id|tag_id| count|
+-------+------+------+
|      1| 30574|167697|
+-------+------+------+
only showing top 1 row

+------+--------+
|tag_id|tag_name|
+------+--------+
|     0|       -|
+------+--------+
only showing top 1 row



# Normalización

Este paso lo debemos aplicar siempre a cada dataset que será usado en el algoritmo ALS.

Sea i un usuario, r(i, j) un rating dado al libro j por el usuario i y r_promedio(i) la calificación promedio del usuario i.

* rating_normalizado = r(i, j) - r_promedio(i)

In [4]:
def normalizeUserRatings(subject: String, feature: String, df: DataFrame): DataFrame = {
    val featureAvg = s"avg($feature)"
    df.groupBy(subject)
        .agg(mean(feature))
        .join(df, Seq(subject))
        .withColumn(feature, col(feature) - col(featureAvg))
        .drop(featureAvg)
}

normalizeUserRatings: (subject: String, feature: String, df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


Asi se ven nuestros ratings normalizados.

In [5]:
val ratings = normalizeUserRatings("user_id", "rating", rawRatings)
ratings.orderBy("user_id", "book_id").show(20)

+-------+-------+--------------------+
|user_id|book_id|              rating|
+-------+-------+--------------------+
|      1|   1180|  0.3333333333333335|
|      1|   4893| -0.6666666666666665|
|      1|   6285|  0.3333333333333335|
|      2|   8034|-0.33333333333333304|
|      2|   8855|   0.666666666666667|
|      2|   9762|-0.33333333333333304|
|      3|   9014|                 0.0|
|      3|   9049|                 0.0|
|      4|   3273|                -2.0|
|      4|   3469|                 1.0|
|      4|   8464|                 1.0|
|      5|   4829| -1.2000000000000002|
|      5|   6646|-0.20000000000000018|
|      5|   6703|-0.20000000000000018|
|      5|   7487|  0.7999999999999998|
|      5|   8072|  0.7999999999999998|
|      6|   7341|                -0.5|
|      6|   8033|                 0.5|
|      7|    585| 0.23684210526315796|
|      7|    910| 0.23684210526315796|
+-------+-------+--------------------+
only showing top 20 rows



ratings = [user_id: int, book_id: int ... 1 more field]


[user_id: int, book_id: int ... 1 more field]

## Etapa 1: Primeras N recomendaciones

Entrenamos nuestro modelo con un bootstrap de 80% para entrenamiento y 20% para validación. En un trabajo futuro se puede usar cross-validation.

In [6]:
val Array(training, test) = ratings.randomSplit(Array(0.8, 0.2))
training.show(1)

+-------+-------+-------------------+
|user_id|book_id|             rating|
+-------+-------+-------------------+
|    148|   1453|0.36734693877551017|
+-------+-------+-------------------+
only showing top 1 row



training = [user_id: int, book_id: int ... 1 more field]
test = [user_id: int, book_id: int ... 1 more field]


[user_id: int, book_id: int ... 1 more field]

Utilizaremos la estrategia drop (eliminar valores nulos) durante la fase de entrenamiento ya que con NaN se puede perder la presición de nuestro sistema, pero cuando se tiene un sistema en producción, debemos utilizar la estrategia NaN, esta nos permite tener recomendaciones por defecto cuando llega un nuevo usuario a nuestro sistema.

In [7]:
val als = new ALS()
    .setMaxIter(5)
    .setRegParam(0.01)
    .setUserCol("user_id")
    .setItemCol("book_id")
    .setRatingCol("rating")
//println(als.explainParams())
val alsModel = als.fit(training)
alsModel.setColdStartStrategy("drop")
val predictions = alsModel.transform(test)
predictions.orderBy("user_id", "book_id").show(20)

+-------+-------+-------------------+-----------+
|user_id|book_id|             rating| prediction|
+-------+-------+-------------------+-----------+
|      6|   8033|                0.5|0.010072887|
|      7|    585|0.23684210526315796|-0.60056484|
|      7|    956|  1.236842105263158| 0.76074624|
|      7|   1464|  1.236842105263158|  0.7185013|
|      7|   1620|  1.236842105263158|  -0.493041|
|      7|   1923|0.23684210526315796| 0.17910463|
|      7|   2084|  1.236842105263158|  0.4816985|
|      7|   2129|  1.236842105263158|   0.681064|
|      7|   2491| -2.763157894736842|-0.79667675|
|      7|   2693|0.23684210526315796|-0.13145779|
|      7|   2766|0.23684210526315796|0.028024688|
|      7|   2969|0.23684210526315796|-0.42445216|
|      7|   3111|0.23684210526315796|  0.3694944|
|      7|   3797| -1.763157894736842|-0.31313896|
|      7|   3884| -0.763157894736842| -1.0044386|
|      7|   4101| -0.763157894736842|-0.86648285|
|      7|   4138| -0.763157894736842|0.049117386|


als = als_3bc6da3eb8e1
alsModel = als_3bc6da3eb8e1
predictions = [user_id: int, book_id: int ... 2 more fields]


[user_id: int, book_id: int ... 2 more fields]

Ahora podemos conocer las recomendaciones top para cada usuario. La instrucción recommendForAllUsers retorna un data frame con array de recomendaciones para cada usuario representado por userId y también el rating de cada libro, recommendForAllItems en cambio, retorna el mismo dataframe, pero con un array del top de usuarios para ese libro.

In [8]:
val userRecs = alsModel.recommendForAllUsers(30)
    .selectExpr("user_id", "explode(recommendations)")
val itemRecommendations = alsModel.recommendForAllItems(30)
    .selectExpr("book_id", "explode(recommendations)")

userRecs.show(20)
itemRecommendations.show(20)

+-------+-----------------+
|user_id|              col|
+-------+-----------------+
|    148| [1400, 1.466075]|
|    148|  [52, 1.4550807]|
|    148|[6634, 1.4195163]|
|    148|[1188, 1.3356676]|
|    148|[6546, 1.3179394]|
|    148|  [39, 1.2452117]|
|    148|[4283, 1.2375112]|
|    148|[2259, 1.2112757]|
|    148| [2717, 1.191325]|
|    148|[3949, 1.1849052]|
|    148|[1703, 1.1716278]|
|    148|[1532, 1.1596256]|
|    148| [350, 1.1317966]|
|    148|  [49, 1.1239188]|
|    148| [4267, 1.119725]|
|    148|[2122, 1.1184484]|
|    148|[1847, 1.1165259]|
|    148|[4195, 1.1140817]|
|    148|[8141, 1.1123798]|
|    148|[1638, 1.1012355]|
+-------+-----------------+
only showing top 20 rows

+-------+------------------+
|book_id|               col|
+-------+------------------+
|   1580|[45764, 2.0233183]|
|   1580|[51381, 2.0078063]|
|   1580| [5928, 1.9380112]|
|   1580| [9442, 1.8996718]|
|   1580|[45050, 1.8908777]|
|   1580|[50827, 1.8440964]|
|   1580|[25066, 1.8284339]|
|   1580|[17

userRecs = [user_id: int, col: struct<book_id: int, rating: float>]
itemRecommendations = [book_id: int, col: struct<user_id: int, rating: float>]


[book_id: int, col: struct<user_id: int, rating: float>]

## Etapa 2: Ratings pesados por tags

Desarmamos el array del dataframe anterior para poder manipular mas fácilmente los datos. Estos nuevos ratings corresponden a las N recomendaciones para cada usuario i, lo siguiente es añadirle el peso del tag a cada rating.

In [9]:
val userTop = userRecs.withColumn("book_id", $"col.book_id")
    .withColumn("rating", $"col.rating")
    .drop($"col")
userTop.show(20)

+-------+-------+---------+
|user_id|book_id|   rating|
+-------+-------+---------+
|    148|   1400| 1.466075|
|    148|     52|1.4550807|
|    148|   6634|1.4195163|
|    148|   1188|1.3356676|
|    148|   6546|1.3179394|
|    148|     39|1.2452117|
|    148|   4283|1.2375112|
|    148|   2259|1.2112757|
|    148|   2717| 1.191325|
|    148|   3949|1.1849052|
|    148|   1703|1.1716278|
|    148|   1532|1.1596256|
|    148|    350|1.1317966|
|    148|     49|1.1239188|
|    148|   4267| 1.119725|
|    148|   2122|1.1184484|
|    148|   1847|1.1165259|
|    148|   4195|1.1140817|
|    148|   8141|1.1123798|
|    148|   1638|1.1012355|
+-------+-------+---------+
only showing top 20 rows



userTop = [user_id: int, book_id: int ... 1 more field]


[user_id: int, book_id: int ... 1 more field]

Hacemos un cruce de los libros con sus repectivos tags.

Para Agregar el tag a cada recomendación, debemos hacer un join con la tabla book_tags.

In [10]:
val userBookTags = userTop.join(book_tags, Seq("book_id")).drop("count")
userBookTags.show(101)

+-------+-------+---------+------+
|book_id|user_id|   rating|tag_id|
+-------+-------+---------+------+
|   2122|    148|1.1184484| 30574|
|   2122|    148|1.1184484|  8717|
|   2122|    148|1.1184484| 11743|
|   2122|    148|1.1184484|  7457|
|   2122|    148|1.1184484| 11557|
|   2122|    148|1.1184484| 23471|
|   2122|    148|1.1184484| 22743|
|   2122|    148|1.1184484| 18367|
|   2122|    148|1.1184484|  7404|
|   2122|    148|1.1184484|  5207|
|   2122|    148|1.1184484| 22034|
|   2122|    148|1.1184484|  2867|
|   2122|    148|1.1184484| 26256|
|   2122|    148|1.1184484| 23931|
|   2122|    148|1.1184484|  3704|
|   2122|    148|1.1184484| 22753|
|   2122|    148|1.1184484| 11590|
|   2122|    148|1.1184484| 30521|
|   2122|    148|1.1184484| 21989|
|   2122|    148|1.1184484|  9221|
|   2122|    148|1.1184484| 12948|
|   2122|    148|1.1184484| 18045|
|   2122|    148|1.1184484|  2277|
|   2122|    148|1.1184484|  1416|
|   2122|    148|1.1184484| 22159|
|   2122|    148|1.1

userBookTags = [book_id: int, user_id: int ... 2 more fields]


[book_id: int, user_id: int ... 2 more fields]

Creamos una nueva relación, la cual contiene el rating de cada usuario i, para algunos tags k, porque no todos los usuarios han calificado libros con cada uno de los tags.

In [11]:
val topTags = userBookTags.groupBy("user_id", "tag_id")
                        .agg(mean("rating"))
                        .withColumnRenamed("avg(rating)", "tag_rating")
topTags.show(20)

+-------+------+------------------+
|user_id|tag_id|        tag_rating|
+-------+------+------------------+
|    148| 10059|1.1251224875450134|
|    148|  9638|1.1251224875450134|
|  18498|  1659|2.8725833892822266|
|  30320| 14370|1.5823676586151123|
|  30320|  7422|1.7068182826042175|
|    731| 22330| 4.746710538864136|
|   7634| 11644|  1.79701566696167|
|  20054| 20731|3.3154056072235107|
|   9147|  9477|2.0902416706085205|
|  11823| 20288|2.1110766530036926|
|  40759|  2867|0.8579235672950745|
|  51944|  8067|1.4814887046813965|
|  26226| 26735|0.4439089596271515|
|  16469| 22753| 1.113128900527954|
|  16469| 18326| 1.113128900527954|
|  12708|  4949|2.4273698329925537|
|  12708| 10644| 2.401083564758301|
|  21319| 20731| 0.784599781036377|
|  12972| 23465|1.6162128448486328|
|  36789| 11798|1.5498590469360352|
+-------+------+------------------+
only showing top 20 rows



topTags = [user_id: int, tag_id: string ... 1 more field]


[user_id: int, tag_id: string ... 1 more field]

Tenemos hasta ahora, un rating de usarios para libros y otro para tags, lo siguiente es sumar ambos ratings, para crear los ratings pesados.

In [12]:
val weightedRecs = userBookTags.join(topTags, Seq("user_id", "tag_id"))
                            .withColumn("rating", $"rating" + $"tag_rating")
                            .drop("tag_rating")
                            .drop("tag_id")
                            .distinct()
weightedRecs.orderBy($"user_id", $"rating".desc).show(20)

+-------+-------+------------------+
|user_id|book_id|            rating|
+-------+-------+------------------+
|     63|   8960|2.2188713550567627|
|    133|    656| 7.914589285850525|
|    202|   6748| 3.547788143157959|
|    217|   3109| 4.188754081726074|
|    367|   7714| 4.756189823150635|
|    577|   9375|3.8534820874532065|
|    762|   5553| 2.784930646419525|
|    958|      8|3.2475767731666565|
|   1006|    350| 6.521520972251892|
|   1150|    250|               0.0|
|   1190|      3| 8.855115175247192|
|   1215|    231|               0.0|
|   1477|   4820|2.5996833244959516|
|   1849|   2050| 3.449760675430298|
|   1854|     30|               0.0|
|   1917|   1319|1.2657529364029565|
|   2028|     30|               0.0|
|   2135|     30|               0.0|
|   2179|     10|               0.0|
|   2300|   1842|5.6391730308532715|
+-------+-------+------------------+
only showing top 20 rows



weightedRecs = [user_id: int, book_id: int ... 1 more field]


[user_id: int, book_id: int ... 1 more field]

## Etapa 3: Recomendación con ratings pesados.

Volvemos a normalizar el rating combinado antes de ejecutar nuevamente el algorimo ALS.

In [13]:
val normRecs = normalizeUserRatings("user_id", "rating", weightedRecs)
//                         .groupBy("user_id", "book_id")
//                         .agg(mean("rating"))
//                         .withColumnRenamed("avg(rating)", "rating")
normRecs.orderBy($"user_id", $"rating".desc).show(50)

+-------+-------+--------------------+
|user_id|book_id|              rating|
+-------+-------+--------------------+
|      1|   4008| 0.14036134878794349|
|      1|   9503|-0.02337874223788583|
|      1|   1110|-0.11698260655005777|
|      2|   5348| 0.09841588139533997|
|      2|   1934|-0.09841588139533997|
|      3|     30|                 0.0|
|      3|    250|                 0.0|
|      3|     10|                 0.0|
|      3|     50|                 0.0|
|      4|   4261| 0.49617784221967076|
|      4|   7214| -0.1891876856486001|
|      4|   9717| -0.3069901565710702|
|      5|   1934|  0.2903935622366379|
|      5|   4009| 0.16281201251817576|
|      5|     11|-0.03631577786945237|
|      5|   7058|-0.06756042538857047|
|      5|     67| -0.0777545711266729|
|      5|   9595| -0.1150482391451008|
|      5|   4633|-0.15286303504637747|
|      6|   3885| 0.10404811799526215|
|      6|   5355|-0.10404811799526215|
|      7|   1934|                 0.0|
|      8|   5553| 0.38112

normRecs = [user_id: int, book_id: int ... 1 more field]


[user_id: int, book_id: int ... 1 more field]

Anteriormente se hizo el entrenamiento paso a paso, pero esta vez simplificamos el entrenamiento a una función.

In [14]:
def makeRecommender(ratings: DataFrame): ALSModel = {
    val Array(training, test) = ratings.randomSplit(Array(0.8, 0.2))
    val als = new ALS()
        .setMaxIter(5)
        .setRegParam(0.01)
        .setUserCol("user_id")
        .setItemCol("book_id")
        .setRatingCol("rating")
    val alsModel = als.fit(training)
    
    alsModel.setColdStartStrategy("drop")
}

makeRecommender: (ratings: org.apache.spark.sql.DataFrame)org.apache.spark.ml.recommendation.ALSModel


Obtenemos nuestro recomendador.

In [15]:
val recommender = makeRecommender(normRecs)

recommender = als_ecb8c7e21286


als_ecb8c7e21286

Creamos las recomendaciones.

In [16]:
def getRecommendations(alsModel: ALSModel, numRecs: Int): DataFrame = {
    val userRecs = alsModel.recommendForAllUsers(numRecs)
        .selectExpr("user_id", "explode(recommendations)")
    
    val userTop = userRecs.withColumn("book_id", $"col.book_id")
    .withColumn("rating", $"col.rating")
    .drop($"col")
    
    userTop
}

getRecommendations: (alsModel: org.apache.spark.ml.recommendation.ALSModel, numRecs: Int)org.apache.spark.sql.DataFrame


Tomamamos ahora las 10 recomendaciones top para cada uno de los usuarios.

In [17]:
val recommendations = getRecommendations(recommender, 10)

recommendations = [user_id: int, book_id: int ... 1 more field]


[user_id: int, book_id: int ... 1 more field]

Guardamos nuestras recomendaciones en nuestro sistema de archivos.

In [22]:
recommendations.write.format("csv").save("./recommendations_by_tags.csv")

lastException: Throwable = null


Finalmente hacemos un join con la tabla libros, para conocer los nombres de los 10 libros top para algunos usuarios. Se puede ver que para algunos usuarios aparecen libros en otros idiomas, por lo cual seria nescesario agregar los idiomas como un nuevo parametro para nuestro recomendador en un futuro.

In [24]:
val recs = recommendations.join(books, Seq("book_id"))
                        .select("original_title", "user_id", "book_id", "rating")
                        .orderBy($"user_id", $"rating".desc)
recs.show(40)

+--------------------+-------+-------+-----------+
|      original_title|user_id|book_id|     rating|
+--------------------+-------+-------+-----------+
|O Demônio e a Srt...|      1|   4008| 0.12814066|
|    The Book of Ruth|      1|   5187|  0.1268632|
|  Benim Adım Kırmızı|      1|   2517| 0.11231489|
|Harry Potter and ...|      1|      3| 0.10356098|
|          Microserfs|      1|   2748|0.102506824|
|                Emma|      1|   6969| 0.08818711|
|Twelfth Night; or...|      1|   1625| 0.08596574|
|      BLEACH―ブリーチ―　1|      1|   2880| 0.08452353|
|Visions of Sugar ...|      1|   6420| 0.08434392|
|By the Shores of ...|      1|   8248| 0.07831517|
|Killers of the Da...|      2|   8952| 0.13937993|
|象の消滅 [Zō no shōme...|      2|   9555| 0.10568179|
|       The Testament|      2|   5348| 0.09179212|
|  A Map of the World|      2|   5205| 0.09175916|
|      The Amber Room|      2|   5369| 0.07434282|
|              Anthem|      2|    667|0.074233696|
|Unfinished Tales ...|      2| 

recs = [original_title: string, user_id: int ... 2 more fields]


[original_title: string, user_id: int ... 2 more fields]

## Evaluación del modelo

Ahora debemos evaluar el desempeño de nuestro modelo. En este caso utilizaremos métricas de ranking, para saber si nuestro modelo es capaz de recomendar un libro que previamente ha sido bien calificado por el usuario.

Note que los ratings en el data frame *predictions* que obtuvimos de la primera etapa, son los ratings reales de nuestro dataset, en el frame *predictions*, la prediciones estan en otra columna.

In [28]:
import org.apache.spark.mllib.evaluation.{RankingMetrics, RegressionMetrics}
import org.apache.spark.sql.functions.{col, expr}

def evaluate(predictions: DataFrame, tag_predictions: DataFrame): Unit = {
    val perUserActual = predictions
        .where("rating > 2.5")
        .groupBy("user_id")
        .agg(expr("collect_set(book_id) as books"))

    val perUserPredictions = predictions
        .orderBy(col("user_id"), col("prediction").desc)
        .groupBy("user_id")
        .agg(expr("collect_list(book_id) as books"))

    val perUserActualvPred = perUserActual.join(perUserPredictions, Seq("user_id"))
        .map(row => (
        row(1).asInstanceOf[Seq[Integer]].toArray,
        row(2).asInstanceOf[Seq[Integer]].toArray.take(15)
        ))
    val ranks = new RankingMetrics(perUserActualvPred.rdd)

    println(s"Mean average Precision = ${ranks.meanAveragePrecision}")
    Seq(1, 3, 5, 8, 10).foreach( k =>
        println(s"Precision at k: $k = ${ranks.precisionAt(k)}")
    )
}

lastException: Throwable = null
evaluate: (predictions: org.apache.spark.sql.DataFrame)Unit


In [30]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
 
def rootMeanSquareError(predictions: DataFrame): Unit = {
    val evaluator = new RegressionEvaluator()
        .setMetricName("rmse")
        .setLabelCol("rating")
        .setPredictionCol("prediction")
    val rmse = evaluator.evaluate(predictions)
    println(s"Root-mean-square error = $rmse")
    print("Global average RMSE = 1.1296, netflix grand prize = 0.8563")
}

rootMeanSquareError: (predictions: org.apache.spark.sql.DataFrame)Unit


# Resultados

En nuestro caso nuestro recomendador acierta bn en la primera predicción, pero apartir de ahi su desempeño cae bastante.

In [29]:
val tag_predictions = recommender.transform(test)
evaluate(predictions, tag_predictions)

Mean average Precision = 1.0
Precision at k: 1 = 1.0
Precision at k: 3 = 0.3333333333333333
Precision at k: 5 = 0.2
Precision at k: 8 = 0.125
Precision at k: 10 = 0.1


tag_predictions = [user_id: int, book_id: int ... 2 more fields]


[user_id: int, book_id: int ... 2 more fields]

En cuando al error cuadrático medio, obtenemos un buen puntaje ya que nuestros ratings ponderados se aproximan el valor del rating real que el usuario ha dado a los libros que le hemos recomendado.

In [31]:
rootMeanSquareError(tag_predictions)

Root-mean-square error = 0.8971356910900724
Global average RMSE = 1.1296, netflix grand prize = 0.8563

# Conclusiones

Nuestro recomendador es bueno prediciendo ratings, pero aun le falta trabajo para hacer buenas recomendaciones, para lo cual debemos probar con mas metadatos y nuevos algoritmos para llegar a una soloción mas óptima.